<h1 style="text-align: center;">Project</h1>
<table>
<tbody>
<tr>
<td style="text-align: left;">Penulis</td>
<td style="text-align: left;">:</td>
<td style="text-align: left;">Moch Rizky Wijaya</td>
</tr>

</tbody>
</table>
<p>&nbsp;</p>
<table>
<tbody>
<tr>
<td style="text-align: left;">Judul Project</td>
<td style="text-align: left;">:</td>
<td style="text-align: left;">Data Modelling</td>
</tr>
<tr>
<td style="text-align: left;">Deskripsi</td>
<td style="text-align: left;">:</td>
<td style="text-align: left;">Project dilakukan dengan collecting data kemudian dimasukkan ke database  RDS. Data yang ada pada database RDS kemudian dimasukkan ke jupyter notebook lagi </td>
</tr>
<tr>
<td style="text-align: left;">Data yang Diambil</td>
<td style="text-align: left;">:</td>
<td style="text-align: left;">Input data dari twitter dengan #matilampulagi #Jakarta</td>
</tr>
</tbody>
</table>
<p>&nbsp;</p>

In [0]:
#tuliskan proses pengambilan data dari twitter atau pembacaan data dari CSV yang sudah disimpan sebelumnya

!pip install tweepy==3.8.0

import tweepy
import csv

from tweepy import OAuthHandler

In [0]:
consumer_key=''
consumer_secret=''

access_token=''
access_secret=''

auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_secret)

api = tweepy.API(auth)

In [0]:
def hitung_jumlah_hashtag(hashtag,n_tweet,date):
  count = 0
  for tweet in tweepy.Cursor(api.search,q=hashtag,date_since=date).items(n_tweet):
    tweet_string = tweet.text
    count =count + tweet_string.count(hashtag)
  return count

In [0]:
jumlah_tweet_yang_diambil = 100
tanggal_mulai = "2019-08-05"
list_hashtag = ["#matilampulagi", "#Jakarta"]

hashtag_counts = {}
for hashtag in list_hashtag:
  hashtag_counts[hashtag] = hitung_jumlah_hashtag(hashtag,jumlah_tweet_yang_diambil,tanggal_mulai)

for k,v in hashtag_counts.items():
  print(k+':'+str(v))

In [0]:
#isi cell ini dengan connection ke database, pembuatan tabel, pengisian data ke tabel, pengambilan data dari database

!pip install pymysql
import pandas as pd
import pymysql

host="database-bigdata.c0hxnu4ehay6.us-east-1.rds.amazonaws.com"
port=3306
dbname="dbbigdata"
user="admin"
password="adminpassword"

conn = pymysql.connect(host, user=user, port=port, passwd =password, db=dbname)

In [0]:
create_tables_m_hashtag="""CREATE TABLE IF NOT EXISTS m_hashtag (
  id_hashtag INT AUTO_INCREMENT, 
  hashtag VARCHAR(100) NULL,
  PRIMARY KEY (id_hashtag));
"""
try:
  with conn.cursor() as cursor:
    cursor.execute(create_tables_m_hashtag)
except Exception as e:
  print(e)
  
create_table_hashtag_count="""CREATE TABLE IF NOT EXISTS hashtag_count (id_hashtag_count INT AUTO_INCREMENT, 
  count VARCHAR(45) NULL,
  id_hashtag INT NOT NULL,
  PRIMARY KEY (id_hashtag_count),
  FOREIGN KEY (id_hashtag)
  REFERENCES m_hashtag (id_hashtag)
  );
"""
try:
  with conn.cursor() as cursor:
    cursor.execute(create_table_hashtag_count)
except Exception as e:
  print(e)

In [0]:
insert_query_1 = "INSERT INTO m_hashtag (hashtag) VALUES (%s)"
insert_query_2 = "INSERT INTO hashtag_count (count,id_hashtag) VALUES (%s,%s)"

try:
  with conn.cursor() as cursor:
    for k,v in hashtag_counts.items():
      cursor.execute(insert_query_1,(k))
      conn.commit()
      
      sql ="SELECT LAST_INSERT_ID()"
      cursor.execute(sql)
      last_insert_id = cursor.fetchone()
      
      cursor.execute(insert_query_2,(v,last_insert_id))
      conn.commit()

except Exception as e:
  print(e)

In [0]:
show_data = "SELECT h.id_hashtag, h.hashtag, hc.count FROM m_hashtag h JOIN hashtag_count hc ON h.id_hashtag = hc.id_hashtag"
records = {}

try:
  with conn.cursor() as cursor:
    cursor.execute(show_data)
    records = cursor.fetchall()
    
    jumlah_baris = cursor.rowcount
    # print jumlah baris 

except Exception as e:
  print("Error")
  print(e)
  
for record in records:
  a,b,c = record
  print(a)
  print(b)
  print(c)

for a,b,c in records:
  print("hashtag %s (id=%s) memiliki count sejumlah %s"%(b,a,c))

Isi cell ini dengan gambar ERD dari database anda dan jelaskan isi dari masing-masing table tersebut. 
Bisa menggunakan https://www.draw.io/ pilih entity relation.

Hasil dapat dilihat melalui link ini
[Gambar ER Diagram](https://drive.google.com/uc?id=1bzkiYUoTEZwJLtkcVXHbYj4v1z3aK--C)

